In [ ]:
import numpy as np
import tensorflow as tf

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

d0 = np.loadtxt("./0.txt", delimiter=" ",dtype=np.float32)
d1 = np.loadtxt("./1.txt", delimiter=" ",dtype=np.float32)

pre_point0 = d0[:,0:6]
next_point0 = d0[:,6:12]
pre_point1 = d1[:,0:6]
next_point1 = d1[:,6:12]

anchor = 100
rate=0.3


train_p_0 = pre_point0[0:int(len(pre_point0)*rate)]
train_n_0 = next_point0[0:int(len(pre_point0)*rate)]
train_p_1 = pre_point1[0:int(len(pre_point1)*rate)]
train_n_1 = next_point1[0:int(len(next_point1)*rate)]


test_p_0 = pre_point0[-1*anchor:]
test_n_0 = next_point0[-1*anchor:]
test_p_1 = pre_point1[-1*anchor:]
test_n_1 = next_point1[-1*anchor:]


In [ ]:
x_pre = tf.placeholder(dtype=tf.float32, shape = [None, 6], name = "pre")
x_next = tf.placeholder(dtype=tf.float32, shape = [None, 6], name = "next")
y = tf.placeholder(dtype=tf.float32, shape = [None, 1], name = "label")

In [ ]:
fc1_pre = tf.layers.dense(inputs=x_pre,
                          units=16,
                          activation=tf.nn.relu,
                          name="fc1_pre")

fc1_next = tf.layers.dense(inputs=x_next,
                           units=64,
                           activation=tf.nn.relu,
                           name="fc1_next")


fc2 = tf.layers.dense(inputs=tf.concat([fc1_pre, fc1_next], 1),
                      units = 128,
                      activation=tf.nn.relu,
                      name="fc2")
fc2_bn = tf.layers.batch_normalization(fc2)

fc3 = tf.layers.dense(inputs=fc2_bn,
                      units = 128,
                      activation=tf.nn.relu,
                      name="fc3")
fc3_bn = tf.layers.batch_normalization(fc3)

fc4 = tf.layers.dense(inputs=fc3_bn,
                      units = 1,
                      activation=tf.nn.sigmoid,
                      name="fc4")



In [ ]:
reg = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-4), tf.trainable_variables())

loss = tf.reduce_sum(tf.square(fc4 -  y))

out = tf.round(fc4)
acc = tf.reduce_mean(tf.cast(tf.equal(out, y), tf.float32))

train_step = tf.train.AdamOptimizer(0.01).minimize(loss+reg)


sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [ ]:
for k in range(300):
    train_loss = []
    train_acc = []
    for i in range(10):
        index_0 = np.random.choice(train_n_0.shape[0], 5000,replace=False)
        index_1 = np.random.choice(train_n_1.shape[0], 5000,replace=True)
        batch_train_p = np.concatenate((train_p_0[index_0],train_p_1[index_1]),0)
        batch_train_n = np.concatenate((train_n_0[index_0],train_n_1[index_1]),0)
        batch_label = np.concatenate((np.zeros(shape=[5000,1]), np.ones(shape=[5000,1])),0)
        id_shuffer = np.random.choice(10000, 10000,replace=False)

        l, _, a, o = sess.run([loss, train_step, acc, out], feed_dict={x_pre:batch_train_p[id_shuffer], x_next:batch_train_n[id_shuffer], y:batch_label[id_shuffer]})
        train_loss.append(l)
        train_acc.append(a)
    print("train_loss = %3f, train_acc = %3f"%(np.mean(train_loss), np.mean(train_acc)))
    test_p = np.concatenate((test_p_0,test_p_1),0)
    test_n = np.concatenate((test_n_0,test_n_1),0)
    label = np.concatenate((np.zeros(shape=[anchor,1]), np.ones(shape=[anchor,1])),0)
    idd_shuffer = np.random.choice(anchor*2, anchor*2, replace=False)
    a,l,ou = sess.run([acc,loss,out] ,feed_dict={x_pre: test_p[idd_shuffer], x_next: test_n[idd_shuffer], y: label[idd_shuffer]})
    print("test_acc = %3f"%a)


In [ ]:
label_3 = label[idd_shuffer]
out_3 = ou
from sklearn.metrics import confusion_matrix
maxtrix = confusion_matrix(label_3, out_3,labels=[1,0])
TP = maxtrix[0][0]
TN = maxtrix[1][1]
FP = maxtrix[1][0]
FN = maxtrix[0][1]
precision = 1.0*  TP  / (TP + FP)
recall = 1.0 * TP/ (TP + FN)
ACC = 1.0*(TP + TN) / (TP + TN + FP + FN)
print('test : TP:%.3f;   TN:%.3f;      FP:%.3f;      FN:%.3f;  precision:%.3f  recall:%.3f  P:%.3f'%(TP, TN, FP, FN, precision, recall, ACC ))



In [ ]:
rate = 0.7

train_p_0 = pre_point0[0:int(len(pre_point0)*rate)]
train_n_0 = next_point0[0:int(len(pre_point0)*rate)]
train_p_1 = pre_point1[0:int(len(pre_point1)*rate)]
train_n_1 = next_point1[0:int(len(next_point1)*rate)]


test_p_0 = pre_point0[-1*anchor:]
test_n_0 = next_point0[-1*anchor:]
test_p_1 = pre_point1[-1*anchor:]
test_n_1 = next_point1[-1*anchor:]


for k in range(300):
    train_loss = []
    train_acc = []
    for i in range(10):
        index_0 = np.random.choice(train_n_0.shape[0], 5000,replace=False)
        index_1 = np.random.choice(train_n_1.shape[0], 5000,replace=True)
        batch_train_p = np.concatenate((train_p_0[index_0],train_p_1[index_1]),0)
        batch_train_n = np.concatenate((train_n_0[index_0],train_n_1[index_1]),0)
        batch_label = np.concatenate((np.zeros(shape=[5000,1]), np.ones(shape=[5000,1])),0)
        id_shuffer = np.random.choice(10000, 10000,replace=False)

        l, _, a, o = sess.run([loss, train_step, acc, out], feed_dict={x_pre:batch_train_p[id_shuffer], x_next:batch_train_n[id_shuffer], y:batch_label[id_shuffer]})
        train_loss.append(l)
        train_acc.append(a)
    print("train_loss = %3f, train_acc = %3f"%(np.mean(train_loss), np.mean(train_acc)))
    test_p = np.concatenate((test_p_0,test_p_1),0)
    test_n = np.concatenate((test_n_0,test_n_1),0)
    label = np.concatenate((np.zeros(shape=[anchor,1]), np.ones(shape=[anchor,1])),0)
    idd_shuffer = np.random.choice(anchor*2, anchor*2, replace=False)
    a,l,ou = sess.run([acc,loss,out] ,feed_dict={x_pre: test_p[idd_shuffer], x_next: test_n[idd_shuffer], y: label[idd_shuffer]})
    print("test_acc = %3f"%a)

In [ ]:
label_7 = label[idd_shuffer]
out_7 = ou
from sklearn.metrics import confusion_matrix
maxtrix = confusion_matrix(label_7, out_7,labels=[1,0])
TP = maxtrix[0][0]
TN = maxtrix[1][1]
FP = maxtrix[1][0]
FN = maxtrix[0][1]
precision = 1.0*  TP  / (TP + FP)
recall = 1.0 * TP/ (TP + FN)
ACC = 1.0*(TP + TN) / (TP + TN + FP + FN)
print('test : TP:%.3f;   TN:%.3f;      FP:%.3f;      FN:%.3f;  precision:%.3f  recall:%.3f  P:%.3f'%(TP, TN, FP, FN, precision, recall, ACC ))




In [ ]:
rate = 0.9

train_p_0 = pre_point0[0:int(len(pre_point0)*rate)]
train_n_0 = next_point0[0:int(len(pre_point0)*rate)]
train_p_1 = pre_point1[0:int(len(pre_point1)*rate)]
train_n_1 = next_point1[0:int(len(next_point1)*rate)]


test_p_0 = pre_point0[-1*anchor:]
test_n_0 = next_point0[-1*anchor:]
test_p_1 = pre_point1[-1*anchor:]
test_n_1 = next_point1[-1*anchor:]


for k in range(300):
    train_loss = []
    train_acc = []
    for i in range(10):
        index_0 = np.random.choice(train_n_0.shape[0], 5000,replace=False)
        index_1 = np.random.choice(train_n_1.shape[0], 5000,replace=True)
        batch_train_p = np.concatenate((train_p_0[index_0],train_p_1[index_1]),0)
        batch_train_n = np.concatenate((train_n_0[index_0],train_n_1[index_1]),0)
        batch_label = np.concatenate((np.zeros(shape=[5000,1]), np.ones(shape=[5000,1])),0)
        id_shuffer = np.random.choice(10000, 10000,replace=False)

        l, _, a, o = sess.run([loss, train_step, acc, out], feed_dict={x_pre:batch_train_p[id_shuffer], x_next:batch_train_n[id_shuffer], y:batch_label[id_shuffer]})
        train_loss.append(l)
        train_acc.append(a)
    print("train_loss = %3f, train_acc = %3f"%(np.mean(train_loss), np.mean(train_acc)))
    test_p = np.concatenate((test_p_0,test_p_1),0)
    test_n = np.concatenate((test_n_0,test_n_1),0)
    label = np.concatenate((np.zeros(shape=[anchor,1]), np.ones(shape=[anchor,1])),0)
    idd_shuffer = np.random.choice(anchor*2, anchor*2, replace=False)
    a,l,ou = sess.run([acc,loss,out] ,feed_dict={x_pre: test_p[idd_shuffer], x_next: test_n[idd_shuffer], y: label[idd_shuffer]})
    print("test_acc = %3f"%a)

In [ ]:
label_9 = label[idd_shuffer]
out_9 = ou
from sklearn.metrics import confusion_matrix
maxtrix = confusion_matrix(label_9, out_9,labels=[1,0])
TP = maxtrix[0][0]
TN = maxtrix[1][1]
FP = maxtrix[1][0]
FN = maxtrix[0][1]
precision = 1.0*  TP  / (TP + FP)
recall = 1.0 * TP/ (TP + FN)
ACC = 1.0*(TP + TN) / (TP + TN + FP + FN)
print('test : TP:%.3f;   TN:%.3f;      FP:%.3f;      FN:%.3f;  precision:%.3f  recall:%.3f  P:%.3f'%(TP, TN, FP, FN, precision, recall, ACC ))





In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,roc_curve

label3 = np.uint8(label_3)
pos_index = np.where(label3 == 1)
pos_distance3 = np.array(out_3)[pos_index[0]]
neg_index = np.where(label3 == 0)
neg_distance3 = np.array(out_3)[neg_index[0]]
plt.figure()
plt.plot(list(range(100)), pos_distance3, 'r*', label='match-distance3')
plt.plot(list(range(100)), neg_distance3, 'gp', label='nonmatch-distance3')
# plt.legend()
auc_ = roc_auc_score(label3, out_3)
fpr3, tpr3, thresholds3 = roc_curve(label3, out_3)

label7= np.uint8(label_7)
pos_index = np.where(label7 == 1)
pos_distance7 = np.array(out_7)[pos_index[0]]
neg_index = np.where(label7 == 0)
neg_distance7 = np.array(out_7)[neg_index[0]]
plt.figure()
plt.plot(list(range(100)), pos_distance7, 'r*', label='match-distance7')
plt.plot(list(range(100)), neg_distance7, 'gp', label='nonmatch-distance7')
# plt.legend()
auc_ = roc_auc_score(label7, out_9)
fpr7, tpr7, thresholds7 = roc_curve(label7, out_7)


label9= np.uint8(label_9)
pos_index = np.where(label9 == 1)
pos_distance9 = np.array(out_9)[pos_index[0]]
neg_index = np.where(label9 == 0)
neg_distance9 = np.array(out_9)[neg_index[0]]
plt.figure()
plt.plot(list(range(100)), pos_distance9, 'r*', label='match-distance9')
plt.plot(list(range(100)), neg_distance9, 'gp', label='nonmatch-distance9')
# plt.legend()
auc_ = roc_auc_score(label9, out_9)
fpr9, tpr9, thresholds7 = roc_curve(label9, out_9)


plt.figure()
plt.plot(fpr3, tpr3, 'r-',label='30_percent')
plt.legend()
plt.plot(fpr7, tpr7, 'b-',label='60_percent')
plt.legend()
plt.plot(fpr9, tpr9, 'g-',label='90_percent')
plt.legend()
plt.title('ROC-curve')
plt.xlabel('FPR', fontsize=12)
plt.ylabel('TPR', fontsize=12)
plt.show()
plt.savefig("./roc.jpg")

